In [1]:
from datasets import load_dataset

In [3]:
! pip install datasets==3.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
fleurs = load_dataset("google/fleurs", "ru_ru")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
fleurs

DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 2562
    })
    validation: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 356
    })
    test: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 775
    })
})

In [5]:
! pip install --no-deps git+https://github.com/salute-developers/GigaAM.git pywer hydra-core kenlm flashlight-text

  Cloning https://github.com/salute-developers/GigaAM.git to /tmp/pip-req-build-y8fb3nw7
  Running command git clone --filter=blob:none --quiet https://github.com/salute-developers/GigaAM.git /tmp/pip-req-build-y8fb3nw7
  Resolved https://github.com/salute-developers/GigaAM.git to commit 6a8b511f753670ed38af6529bb89bbdc2191ba6a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
  Created wheel for gigaam: filename=gigaam-0.1.0-py3-none-any.whl size=22491 sha256=6e71df7ffc4145e6bf434d1187f58c0d70595a576b4edace701cfb105941d963
  Stored in directory: /tmp/pip-ephem-wheel-cache-34xiqpfz/wheels/22/24/9c/7ab61a789d64bf3684c9929ae3128b

In [4]:
import os
import torch
import torch.nn as nn
import gigaam

In [5]:
from gigaam import GigaAMASR

CACHE_DIR = os.path.expanduser("~/.cache/gigaam")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name, model_path = gigaam._download_model('ctc', CACHE_DIR)

ckpt = torch.load(model_path, map_location='cpu', weights_only=False)

ckpt["cfg"].encoder.flash_attn = False
model = GigaAMASR(ckpt['cfg'])

model.load_state_dict(ckpt["state_dict"], strict=False)
model = model.eval()

if device.type != "cpu":
  model.encoder = model.encoder.half()

model = model.to(device)

In [6]:
from torch import Tensor
import torchaudio
from typing import Tuple

class SpecScaler(nn.Module):
    """
    Module that applies logarithmic scaling to spectrogram values.
    This module clamps the input values within a certain range and then applies a natural logarithm.
    """

    def forward(self, x: Tensor) -> Tensor:
        return torch.log(x.clamp_(1e-9, 1e9))


class FeatureExtractor(nn.Module):
    """
    Module for extracting Log-mel spectrogram features from raw audio signals.
    This module uses Torchaudio's MelSpectrogram transform to extract features
    and applies logarithmic scaling.
    """

    def __init__(self, sample_rate: int, features: int):
        super().__init__()
        self.hop_length = sample_rate // 100
        self.featurizer = nn.Sequential(
            torchaudio.transforms.MelSpectrogram(
                sample_rate=sample_rate,
                n_fft=sample_rate // 40,
                win_length=sample_rate // 40,
                hop_length=self.hop_length,
                n_mels=features,
            ),
            SpecScaler(),
        )

    def out_len(self, input_lengths: Tensor) -> Tensor:
        """
        Calculates the output length after the feature extraction process.
        """
        return input_lengths.div(self.hop_length, rounding_mode="floor").add(1).long()

    def forward(self, input_signal: Tensor, length: Tensor) -> Tuple[Tensor, Tensor]:
        """
        Extract Log-mel spectrogram features from the input audio signal.
        """
        return self.featurizer(input_signal), self.out_len(length)

In [7]:
from torch.utils.data import Dataset, DataLoader
from typing import Dict

class AudioDataset(Dataset):
    """
    Датасет для загрузки аудиофайлов и транскрипций

    Ожидаемый формат данных:
    - manifest_path: путь к JSON файлу с метаданными
    - Формат JSON: [{"audio_path": "path/to/audio.wav", "text": "транскрипция"}, ...]
    """

    def __init__(self, dataset, preprocessor):
       self.dataset = dataset
       self.preprocessor = preprocessor

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx: int) -> Dict:
        sample = self.dataset[idx]

        # mel_spec_signal, signal_len = self.preprocessor(
        #    torch.tensor(sample['audio']['array'], dtype=torch.float32),
        #    torch.tensor(sample['num_samples'], dtype=torch.int32)
        # )

        # return {
        #     'audio': mel_spec_signal,
        #     'num_samples': signal_len,
        #     'transcription': sample['transcription'],
        # }

        return {
            'audio': torch.tensor(sample['audio']['array'], dtype=torch.float32),
            'num_samples': torch.tensor(sample['num_samples'], dtype=torch.int32),
            'transcription': sample['transcription'],
        }

In [8]:
def collate_fn(batch):
    # Для полей с разной длиной (например, audio) нужно добавить паддинг
    audio = [item['audio'] for item in batch]
    audio_padded = torch.nn.utils.rnn.pad_sequence(audio, batch_first=True)

    return audio_padded, torch.stack([item['num_samples'] for item in batch]), [item['transcription'] for item in batch]

In [9]:
dataset = AudioDataset(fleurs['train'], preprocessor=FeatureExtractor(sample_rate=16000, features=64))

In [10]:
train_loader = DataLoader(
            dataset,
            batch_size=1,
            shuffle=True,
            num_workers=1,
            collate_fn=collate_fn,
        )

In [27]:
for batch in train_loader:
  audio, num_samples, texts = batch
  print(audio)
  print(num_samples)
  print(texts)
  break

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e47a5d5cf40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0002, -0.0001, -0.0002]])
tensor([148800], dtype=torch.int32)
['однако люди которые знают немного испанский язык могут поспешно заключить что португальский язык достаточно схож и его можно не учить отдельно']


In [11]:
def get_gigaam_logprobs(model, wav_batch, wav_lengths, return_transcriptions=False):
    wav_batch = wav_batch.to(model._device)
    wav_lengths = wav_lengths.to(model._device)

    encoded, encoded_len = model.forward(wav_batch, wav_lengths)

    logprobs = model.head(encoded)

    if return_transcriptions:
        transcriptions = model.decoding.decode(model.head, encoded, encoded_len)
        return logprobs, encoded_len, transcriptions
    else:
        return logprobs, encoded_len

In [12]:
from pickle import encode_long
def compute_ctc_loss(wav_batch, wav_lengths, transcripts, transcript_lengths):
        # Получаем логиты от модели
        logprobs, encoded_len = get_gigaam_logprobs(model, wav_batch, wav_lengths)

        # print(f"[DEBUG] {encoded_len}")
        encoded_len = tuple(encoded_len.numpy())
        # print(f"[DEBUG] {encoded_len}")

        # CTCLoss требует логиты в формате (T, N, C)
        # Где T - временная длина, N - размер батча, C - число классов
        logprobs = logprobs.transpose(0, 1)  # Теперь форма (T, N, C)

        # Инициализируем CTC Loss
        # ctc_loss = nn.CTCLoss(blank=self.model.decoding.blank_id, reduction='mean', zero_infinity=True)
        #! Here can be an error
        BLANK_IDX = 33
        ctc_loss = nn.CTCLoss(blank=BLANK_IDX, reduction='mean', zero_infinity=True)

        # print(f"[DEBUG] logprobs len {len(logprobs)}")
        # print(f"[DEBUG] transcription length {len(transcripts[0])}")
        # Вычисляем потерю
        loss = ctc_loss(
            logprobs,           # (T, N, C)
            transcripts,        # (N, S) -> целочисленные индексы
            encoded_len,        # (N,) -> длины выходных последовательностей
            transcript_lengths  # (N,) -> длины целевых последовательностей
        )

        return loss

In [33]:
model.decoding.blank_id

33

In [36]:
model.decoding.tokenizer.vocab

[' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']

In [13]:
model_vocab = {sym: idx for idx, sym in enumerate(model.decoding.tokenizer.vocab)}

In [21]:
import re

def preprocess_text(text):
    """
    Предобрабатывает текст по заданным правилам:
    1. Оставляет только символы русского алфавита и пробелы
    2. Заменяет дефисы на пробелы
    3. Заменяет латинские буквы на близкие русские
    4. Заменяет арабские и римские цифры на слова
    """

    # Словарь замены латинских букв на русские
    latin_to_russian = {
        'a': 'а', 'A': 'А',
        'b': 'б', 'B': 'Б',
        'c': 'к', 'C': 'К',
        'd': 'д', 'D': 'Д',
        'e': 'е', 'E': 'Е',
        'f': 'ф', 'F': 'Ф',
        'g': 'г', 'G': 'Г',
        'h': 'х', 'H': 'Х',
        'i': 'и', 'I': 'И',
        'j': 'й', 'J': 'Й',
        'k': 'к', 'K': 'К',
        'l': 'л', 'L': 'Л',
        'm': 'м', 'M': 'М',
        'n': 'н', 'N': 'Н',
        'o': 'о', 'O': 'О',
        'p': 'п', 'P': 'П',
        'q': 'к', 'Q': 'К',
        'r': 'р', 'R': 'Р',
        's': 'с', 'S': 'С',
        't': 'т', 'T': 'Т',
        'u': 'у', 'U': 'У',
        'v': 'в', 'V': 'В',
        'w': 'в', 'W': 'В',
        'x': 'кс', 'X': 'Кс',
        'y': 'у', 'Y': 'У',
        'z': 'з', 'Z': 'З'
    }

    # Словарь для замены арабских цифр
    digit_to_word = {
        '0': 'ноль',
        '1': 'один',
        '2': 'два',
        '3': 'три',
        '4': 'четыре',
        '5': 'пять',
        '6': 'шесть',
        '7': 'семь',
        '8': 'восемь',
        '9': 'девять'
    }

    # Словарь для замены римских цифр
    roman_to_word = {
        'I': 'один', 'II': 'два', 'III': 'три', 'IV': 'четыре', 'V': 'пять',
        'VI': 'шесть', 'VII': 'семь', 'VIII': 'восемь', 'IX': 'девять', 'X': 'десять',
        'XI': 'одиннадцать', 'XII': 'двенадцать', 'XIII': 'тринадцать',
        'XIV': 'четырнадцать', 'XV': 'пятнадцать', 'XVI': 'шестнадцать',
        'XVII': 'семнадцать', 'XVIII': 'восемнадцать', 'XIX': 'девятнадцать',
        'XX': 'двадцать'
    }

    # Приводим текст к нижнему регистру для удобства обработки
    text = text.lower()

    # Заменяем римские цифры (обрабатываем сначала перед другими преобразованиями)
    for roman, word in sorted(roman_to_word.items(), key=lambda x: len(x[0]), reverse=True):
        text = re.sub(r'\b' + roman + r'\b', word, text, flags=re.IGNORECASE)

    # Заменяем латинские буквы на русские
    for latin, russian in latin_to_russian.items():
        text = text.replace(latin, russian)

    # Заменяем арабские цифры
    for digit, word in digit_to_word.items():
        text = text.replace(digit, word)

    # Заменяем дефисы на пробелы
    text = text.replace('-', ' ')

    # Удаляем все символы, кроме русских букв и пробелов
    text = re.sub(r'[^а-я\s]', '', text, flags=re.IGNORECASE)

    # Убираем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [15]:
from typing import List, Any

def get_texts_idxs(texts: List[str]) -> torch.Tensor:
  texts_idxs = []
  for text in texts:
    print(f"[DEBUG] {text}")

    text = preprocess_text(text)

    print(f"[DEBUG] preprocessed text: {text}")

    text_idxs = [model_vocab[sym] for sym in text]
    texts_idxs.append(text_idxs)

  return torch.tensor(texts_idxs, dtype=torch.int)

In [27]:
for batch in train_loader:
  audio, num_samples, texts = batch

  transcript_lengths=(len(sample) for sample in texts)
  loss = compute_ctc_loss(
            audio,
            num_samples,
            get_texts_idxs(texts),
            transcript_lengths=tuple(transcript_lengths)
        )
  print(loss)
  break

[DEBUG] карно является знаменитым учителем английского со спорной репутацией он преподавал в учебных заведениях современное образование и королевская слава и заявлял что на пике карьеры у него было 9000 учащихся
[DEBUG] preprocessed text: карно является знаменитым учителем английского со спорной репутацией он преподавал в учебных заведениях современное образование и королевская слава и заявлял что на пике карьеры у него было девятьнольнольноль учащихся
tensor(1.2369, grad_fn=<MeanBackward0>)
